## Imports

In [1]:
import numpy as np
from stable_baselines3 import SAC

from utils.data_loader import load_price_data
from envs.bess_env import BatteryEnv

## Load hourly day-ahead price data

In [2]:
csv_path = "../data/raw/dayahead_2024_11.csv"

df, price_series, timestamps = load_price_data(
    csv_path,
    resolution="1h"   # or "15min" for finer simulation
)

df.head()

,Nicht Erneuerbar,Erneuerbar,Day Ahead Auktion
Datum (MEZ),,,
2024-11-01 00:00:00+01:00,18141.163535,28211.565575,64.1975
2024-11-01 01:00:00+01:00,16762.893740,27801.153758,56.2275
2024-11-01 02:00:00+01:00,15753.801843,28020.887427,49.9025
2024-11-01 03:00:00+01:00,15673.267653,27061.248645,44.9250
2024-11-01 04:00:00+01:00,14630.770720,27138.154085,50.1650


## Initialize BatteryEnv with real timestamps

In [3]:
env = BatteryEnv(
    price_series=price_series,
    timestamps=timestamps,
    dt_hours=1.0,          # corresponds to 1H resolution
    capacity_kWh=100.0,
    p_max_kW=50.0,
    price_unit="EUR_per_MWh",
    price_sigma_rel=0.10,
    use_simple_cycle_count=True,
    deg_cost_per_EFC=120.0,
)

obs, info = env.reset()
obs

array([ 0.42787847,  1.        ,  0.        ,  1.        , -0.85876393,
        0.5123714 ,  0.09744313,  0.        ,  0.        ], dtype=float32)

## Test with a small SAC agent

In [4]:
model = SAC("MlpPolicy", env, verbose=1)

model.learn(
    total_timesteps=2_000,
    log_interval=1,
    progress_bar=True
)

print("Training completed.")

 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1,996/2,000  [ 0:00:51 < 0:00:01 , 37 it/s ]

Training completed.
